In [1]:
#1.Q1
import pandas as pd  
  
# 读取csv文件  
df = pd.read_csv('C:\\Users\\86180\\Desktop\\数据分析实验四\\data2.csv')  

# 检查哪些特征含有缺失值  
missing_values = df.isnull().sum()  
print("Features with missing values:")  
print(missing_values[missing_values > 0])  # 只显示含有缺失值的特征  
  
# 删除所有含空缺值的行  
df = df.dropna()  



Features with missing values:
node-caps      8
breast-quad    1
dtype: int64


In [2]:
# 1.Q2.（10%）当前数据表未能正确处理部分数据值的文本与日期表示类型
# 检查tumor-size列的值分布  
print("Value counts for tumor-size:")  
print(df['tumor-size'].value_counts())  
  
# 检查inv-nodes列的值分布  
print("\nValue counts for inv-nodes:")  
print(df['inv-nodes'].value_counts())  

# 修正tumor-size列中的异常值 
replacements = {
    '14-Oct':'10-14',
    '9-May':'5-9',
    '5-Mar':'3-5',
    '8-Jun':'6-8',
    '11-Sep':'9-11',
    '14-Dec':'12-14'
}
# 使用replace方法替换值  
df['inv-nodes'].replace(replacements, inplace=True)  
df['tumor-size'].replace(replacements, inplace=True) 

# 检查tumor-size列的值分布  
print("Value counts for tumor-size:")  
print(df['tumor-size'].value_counts())  
  
# 检查inv-nodes列的值分布  
print("\nValue counts for inv-nodes:")  
print(df['inv-nodes'].value_counts())  


Value counts for tumor-size:
tumor-size
30-34     57
25-29     51
20-24     48
15-19     29
14-Oct    28
40-44     22
35-39     19
0-4        8
50-54      8
9-May      4
45-49      3
Name: count, dtype: int64

Value counts for inv-nodes:
inv-nodes
0-2       209
5-Mar      34
8-Jun      17
11-Sep      7
15-17       6
14-Dec      3
24-26       1
Name: count, dtype: int64
Value counts for tumor-size:
tumor-size
30-34    57
25-29    51
20-24    48
15-19    29
10-14    28
40-44    22
35-39    19
0-4       8
50-54     8
5-9       4
45-49     3
Name: count, dtype: int64

Value counts for inv-nodes:
inv-nodes
0-2      209
3-5       34
6-8       17
9-11       7
15-17      6
12-14      3
24-26      1
Name: count, dtype: int64


In [3]:
#1.Q3
# 读取包含特征映射的Excel文件  
mapping_df = pd.read_excel('C:\\Users\\86180\\Desktop\\数据分析实验四\\variables.xlsx') 
# 初始化索引与属性值的对应关系字典  
ind2val = {} 

# 遍历映射DataFrame，为每个特征创建映射关系  
flag = 0
for index, row in mapping_df.iterrows():  
    feature_name = row['Variable Name']  #这一列包含了特征名  
    value_to_index_pairs = row['Description'].split(',')  # 映射之间用逗号分隔    
  
 # 遍历每个值到索引的配对  
    for pair in value_to_index_pairs:  
        pair= pair.strip()  
        # 创建特征名和具体值的组合作为字典的键  
        key = f'{feature_name}={pair}'  
          
        # 添加映射到ind2val字典  
        ind2val[key] = flag 
        flag = flag+1
  
print(ind2val) 

# 遍历数据集的每一列  
for column in df.columns:  
    # 检查该列是否在映射字典的特征名中  
    for key, value in ind2val.items():  
        feature_name, feature_value = key.split('=')  
        if column == feature_name:  
            # 替换该列中的值为字典中对应的索引  
            df[column] = df[column].replace(feature_value, value)  
#print(df.head())

{'Class=no-recurrence-events': 0, 'Class=recurrence-events': 1, 'age=10-19': 2, 'age=20-29': 3, 'age=30-39': 4, 'age=40-49': 5, 'age=50-59': 6, 'age=60-69': 7, 'age=70-79': 8, 'age=80-89': 9, 'age=90-99': 10, 'menopause=lt40': 11, 'menopause=ge40': 12, 'menopause=premeno': 13, 'tumor-size=0-4': 14, 'tumor-size=5-9': 15, 'tumor-size=10-14': 16, 'tumor-size=15-19': 17, 'tumor-size=20-24': 18, 'tumor-size=25-29': 19, 'tumor-size=30-34': 20, 'tumor-size=35-39': 21, 'tumor-size=40-44': 22, 'tumor-size=45-49': 23, 'tumor-size=50-54': 24, 'tumor-size=55-59': 25, 'inv-nodes=0-2': 26, 'inv-nodes=3-5': 27, 'inv-nodes=6-8': 28, 'inv-nodes=9-11': 29, 'inv-nodes=12-14': 30, 'inv-nodes=15-17': 31, 'inv-nodes=18-20': 32, 'inv-nodes=21-23': 33, 'inv-nodes=24-26': 34, 'inv-nodes=27-29': 35, 'inv-nodes=30-32': 36, 'inv-nodes=33-35': 37, 'inv-nodes=36-39': 38, 'node-caps=yes': 39, 'node-caps=no': 40, 'deg-malig=1': 41, 'deg-malig=2': 42, 'deg-malig=3': 43, 'breast=left': 44, 'breast=right': 45, 'breast-q

In [4]:
 
import numpy as np  
   
# 将DataFrame转换为列表的列表形式，每个内部列表代表一个事务  
def convert_df_to_transactions(df):  
    transactions = df.values.tolist()    
    return transactions  
  
# 计算项集的支持度  
def calculate_support(transactions, itemset):  
    count = sum(set(itemset).issubset(transaction) for transaction in transactions)  
    return count / len(transactions)  
  
def generate_candidate_one_set(Lk_minus_1, all_items):  
    candidates = set()  
      
    # 遍历Lk_minus_1中的每个项  
    for item in Lk_minus_1:  
        # 生成新的候选项，通过添加不在Lk_minus_1中但在all_items中的项  
        remaining_items = all_items - Lk_minus_1  
        for new_item in remaining_items:  
            # 创建一个新的候选项，通过在(k-1)项集中添加new_item  
            candidate = Lk_minus_1 | frozenset([new_item])  
            candidates.add(candidate)  
      
    return candidates
# Apriori算法实现  
def apriori(transactions, min_support=0.4):  
    if not isinstance(transactions, list):  
        transactions = convert_df_to_transactions(transactions)  # 确保transactions是列表的列表  
    #print(transactions)
    
    all_items = set(item for transaction in transactions for item in transaction)  
 
    # 1. 初始化频繁1项集  
    C1 = [frozenset([item]) for item in all_items]  
    F1 = [itemset for itemset in C1 if calculate_support(transactions, itemset) >= min_support]  
    k = 2  
    frequent_itemsets = F1[:]  
    #print(frequent_itemsets[-1])  
    # 2. 迭代生成并测试候选项集  
    while True:  
        Ck = set()  # 初始化候选项集集合  
  
        # 遍历频繁(k-1)项集列表  
        for Lk_minus_1 in frequent_itemsets:  
            if len(Lk_minus_1) == k-1:  # 确保只处理长度为k-1的项集  
                # 为每个频繁(k-1)项集生成候选项集  
                #print(Lk_minus_1)
                candidate_sets = generate_candidate_one_set(Lk_minus_1, all_items)  
                Ck.update(candidate_sets)  # 将生成的候选项集添加到Ck中  
         
        # 计算候选项集的支持度  
        Ck_counts = {itemset: calculate_support(transactions, itemset) for itemset in Ck}  
          
        # 保留支持度大于或等于min_support的项集  
        Fk = [itemset for itemset, count in Ck_counts.items() if count >= min_support]  
          
        # 如果没有新的频繁项集，则停止  
        if len(Fk) == 0:  
            break  
          
        # 更新频繁项集列表  
        frequent_itemsets.extend(Fk)  
        k += 1  
      
    # 转换为DataFrame格式（如果需要的话）  
    frequent_itemsets_with_support = [(itemset, calculate_support(transactions, itemset)) for itemset in frequent_itemsets]  
    frequent_itemsets_df = pd.DataFrame(frequent_itemsets_with_support, columns=['Itemsets', 'Support'])  
      
    return frequent_itemsets_df
  
# 示例用法  
transactions = convert_df_to_transactions(df)   
frequent_itemsets = apriori(transactions, min_support=0.4)  
print(frequent_itemsets)


           Itemsets   Support
0               (0)  0.707581
1               (1)  0.501805
2               (2)  0.465704
3              (12)  0.447653
4              (13)  0.541516
5              (26)  0.754513
6              (40)  0.797834
7              (44)  0.527076
8              (45)  0.480144
9              (52)  0.776173
10         (40, 44)  0.418773
11         (40, 13)  0.425993
12         (26, 13)  0.404332
13         (26, 52)  0.649819
14          (0, 52)  0.592058
15         (44, 52)  0.415162
16         (40, 26)  0.722022
17         (26, 44)  0.404332
18          (0, 26)  0.599278
19         (52, 13)  0.404332
20          (0, 40)  0.617329
21         (40, 52)  0.675090
22      (0, 26, 52)  0.530686
23      (0, 40, 52)  0.545126
24     (40, 26, 52)  0.635379
25      (40, 0, 26)  0.577617
26  (0, 26, 40, 52)  0.523466


In [5]:
#2.Q2.（20%）基于提取出的频繁项集，以最小置信度阈值为0.75，提取形如X->{0}的强关联规则，并分别输出它们的置信度和提升度。
#print(frequent_itemsets['Itemsets'])

def calculate_support(item_or_set, df):  
    # 查找单个项或项集的支持度  
    return df[df['Itemsets'].apply(lambda x: item_or_set.issubset(x))]['Support'].iloc[0] if not df.empty else 0  
  
def calculate_confidence(antecedent, consequent, df):      
    # 查找X ∪ {item}的支持度  
    union_support = calculate_support(antecedent | {consequent}, df)  
    # 查找X的支持度  
    antecedent_support = calculate_support(antecedent, df)  
    # 计算置信度  
    return union_support / antecedent_support if antecedent_support > 0 else 0  
  
def calculate_lift(antecedent, consequent, df):  
    # 查找X的支持度  
    antecedent_support = calculate_support(antecedent, df)  
    # 查找{item}的支持度  
    consequent_support = calculate_support({consequent}, df)  
    # 查找X ∪ {item}的支持度  
    union_support = calculate_support(antecedent | {consequent}, df)  
    # 计算提升度  
    return union_support / (antecedent_support * consequent_support) if (antecedent_support > 0 and consequent_support > 0) else 0  
  
  
def generate_strong_rules(df, min_confidence=0.75):  
    strong_rules = []  
    for index, row in df.iterrows():  
        itemset = row['Itemsets']  
        #support = row['Support']  
        # 只考虑长度大于1的项集  
        if len(itemset) > 1:  
             for item in itemset:
                if item == 0:
                # 生成形如 X->{0} 的规则，其中 X = itemset - {0}  ,
                    antecedent = set(tuple(sorted(itemset - {item})))                 
                # 计算置信度  
                    confidence = calculate_confidence(antecedent, item, df)  
                    lift = calculate_lift(antecedent, item, df)
                # 如果置信度大于或等于最小置信度，则添加规则  
                    if confidence >= min_confidence:  
                        rule = (antecedent, item, confidence,lift)  
                        strong_rules.append(rule)  
    return strong_rules 
the_rules = generate_strong_rules(frequent_itemsets, min_confidence=0.75)
for rule in the_rules:
    print(rule)

({52}, 0, 0.7627906976744185, 1.0780256288561936)
({26}, 0, 0.7942583732057417, 1.1224978029489308)
({40}, 0, 0.7737556561085973, 1.0935220241942933)
({26, 52}, 0, 0.8166666666666667, 1.1541666666666668)
({40, 52}, 0, 0.8074866310160427, 1.1411928407726726)
({40, 26}, 0, 0.7999999999999999, 1.1306122448979592)
({40, 26, 52}, 0, 0.8238636363636364, 1.1643378942486085)


In [6]:
#2.Q3.（10%）参考ind2val中索引与属性值的对应关系，对以上频繁项集和关联规则结果进行简要分析和总结。

#字典ind2val中含有数字索引和病人特征的键值对，现在打算将这些强关联规则返回为原来的特征关联，方便分析

# 创建逆映射字典  
inverse_dict = {v: k for k, v in ind2val.items()}  

replaced_list = []  
for rule in the_rules:  
    # 替换集合
    antecedent = set()  # 假设集合中只有一个元素 
    for element in rule[0]:
        antecedent.add(inverse_dict.get(element,element))          
    # 替换整数  
    item = inverse_dict.get(rule[1], rule[1])  # 使用get方法来避免KeyError，如果整数没有替换值则保持不变  
    # 构建新的元组并添加到新列表中  
    replaced_list.append((antecedent, item, rule[2], rule[3]))  

for x in replaced_list:
    print(x)

({'irradiat=no'}, 'Class=no-recurrence-events', 0.7627906976744185, 1.0780256288561936)
({'inv-nodes=0-2'}, 'Class=no-recurrence-events', 0.7942583732057417, 1.1224978029489308)
({'node-caps=no'}, 'Class=no-recurrence-events', 0.7737556561085973, 1.0935220241942933)
({'irradiat=no', 'inv-nodes=0-2'}, 'Class=no-recurrence-events', 0.8166666666666667, 1.1541666666666668)
({'irradiat=no', 'node-caps=no'}, 'Class=no-recurrence-events', 0.8074866310160427, 1.1411928407726726)
({'inv-nodes=0-2', 'node-caps=no'}, 'Class=no-recurrence-events', 0.7999999999999999, 1.1306122448979592)
({'irradiat=no', 'inv-nodes=0-2', 'node-caps=no'}, 'Class=no-recurrence-events', 0.8238636363636364, 1.1643378942486085)
